In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

#tb.loc[tb['Instrument Classification (ISO 10962 CFI code)'].isin(['JFTXFP'])]
#tb.plot.hist(tb.groupby(['Trading platform']).Price.agg(['mean', max]))
#tb.describe()
#print(tb.shape)

df = pd.read_feather('C:/Users/asus/Documents/rts27.feather')

#Columns deemed unneccessary for this task
df2 = df.drop(['file_name','designated_market_makers','legal_entity_name', 'systematic_internaliser_flag'], axis = 1)


In [23]:
#Columns deemed unneccessary for this task
#df = df.drop(['file_name','designated_market_makers','legal_entity_name', 'systematic_internaliser_flag'], axis = 1)
df = df2

#Filters the code to FX forwards only
#df = df.loc[(df['source_instrument_desc']).astype(str).str.contains("Forward")]
df = df.loc[(df['instrument_classification']).astype(str).str.contains("JF") | (df['source_instrument_desc']).astype(str).str.contains("Forward")]
#Drops the columns of which half of the fields have no data.
#df = df.dropna(axis=1, thresh=2000000)
for col in df.columns:
    #print(col, df[col].dtypes)
    if df[col].dtypes == 'int'or df[col].dtypes == 'float64' or df[col].dtypes == 'datetime64[ns]':
        df[col].fillna(0,  inplace=True)
    else: 
        df[col].fillna("NaN",  inplace=True)
        

C:\Users\asus\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,id,bank_name,date_of_trading_day,instrument_identifier,instrument_classification,order_request_quotes_received,executed_transactions,total_value_of_transactions,order_request_quotes_can_wdrwn,order_request_quotes_modified,...,median_transaction_size_eur,median_transaction_size_gbp,median_size_request_quotes_eur,median_size_request_quotes_gbp,smartie_instrument_name,data_quality_flag,egb_eligible_flag,year_quarter,business,central_bank_isin_flag
59,73994359,Credit Suisse,2018-10-16,NaN,NaN,2.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,2018 Q4,NaN,NaN
61,73994361,Credit Suisse,2018-10-16,NaN,NaN,2.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,2018 Q4,NaN,NaN
72,73994372,Credit Suisse,2018-10-16,NaN,NaN,18.0,0.0,0.0,6.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,2018 Q4,NaN,NaN
73,73994373,Credit Suisse,2018-10-16,NaN,NaN,2.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,2018 Q4,NaN,NaN
100,67771594,Morgan Stanley,2018-09-13,EZ43HP8WC2Y3,JFTXFC,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,2018 Q3,NaN,NaN
101,67771595,Morgan Stanley,2018-09-28,EZ24NVRGSWM1,JFTXFP,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,2018 Q3,NaN,NaN
102,67771596,Morgan Stanley,2018-09-06,NaN,JFTXFP,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,2018 Q3,NaN,NaN
103,67771597,Morgan Stanley,2018-07-13,EZ6HV8SZ0GB1,JFTXFP,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,2018 Q3,NaN,NaN
104,67771598,Morgan Stanley,2018-08-31,NaN,JFTXFP,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,2018 Q3,NaN,NaN
105,67771599,Morgan Stanley,2018-07-20,EZW2B7BGWGS3,JFTXFP,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,2018 Q3,NaN,NaN


In [24]:
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Select target
y = df['date_created']
# To keep things simple, we'll use only numerical predictors
df = df.drop(['date_created'], axis=1)
X = df

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

y.head(10)

59    2019-05-13 14:24:39
61    2019-05-13 14:24:39
72    2019-05-13 14:24:39
73    2019-05-13 14:24:39
100   2019-01-15 15:01:55
101   2019-01-15 15:01:55
102   2019-01-15 15:01:55
103   2019-01-15 15:01:55
104   2019-01-15 15:01:55
105   2019-01-15 15:01:55
Name: date_created, dtype: datetime64[ns]

In [26]:
from sklearn.preprocessing import LabelEncoder
import bisect

# Get list of categorical variables
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

#print("Categorical variables:")
#print(object_cols)

# Make copy to avoid changing original data 
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Apply label encoder to each column with categorical data
le = LabelEncoder()
for col in object_cols:
    le = LabelEncoder()
    le.fit(df[col])
    label_X_train[col] = le.transform(X_train[col])
    label_X_valid[col] = le.transform(X_valid[col])
    
label_X_train.head(20)

,id,bank_name,date_of_trading_day,instrument_identifier,instrument_classification,order_request_quotes_received,executed_transactions,total_value_of_transactions,order_request_quotes_can_wdrwn,order_request_quotes_modified,...,median_transaction_size_eur,median_transaction_size_gbp,median_size_request_quotes_eur,median_size_request_quotes_gbp,smartie_instrument_name,data_quality_flag,egb_eligible_flag,year_quarter,business,central_bank_isin_flag
8077196,7180631,7,2018-03-26,36768,14,10.0,2.0,2.640000e+08,8.0,0.0,...,1.060923e+08,9.279438e+07,2.009323e+07,1.757469e+07,0,0,0,0,0,0
4205086,70770209,10,2018-10-01,57028,12,4.0,0.0,0.000000e+00,3.0,0.0,...,0.000000e+00,0.000000e+00,6.000000e+06,0.000000e+00,0,0,0,3,0,0
3521704,81746350,0,2018-10-29,129417,12,32.0,0.0,0.000000e+00,3.0,0.0,...,0.000000e+00,0.000000e+00,1.000000e+06,8.898000e+05,0,0,0,3,0,0
2338297,8911643,8,2018-02-28,42105,12,1.0,0.0,0.000000e+00,0.0,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0,0,0,0,0
12823767,25967823,7,2018-06-01,50883,14,10.0,0.0,0.000000e+00,10.0,0.0,...,0.000000e+00,0.000000e+00,2.149386e+06,1.881005e+06,0,0,0,1,0,0
1271687,46193545,3,2018-09-28,5680,12,0.0,0.0,0.000000e+00,0.0,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0,0,2,0,0
1952011,70259724,7,2018-11-01,83876,14,9.0,3.0,1.676318e+06,6.0,0.0,...,5.100000e+05,4.481113e+05,5.100000e+05,4.481113e+05,0,0,0,3,0,0
11613657,17783091,13,2018-02-13,40230,12,13.0,0.0,0.000000e+00,0.0,0.0,...,0.000000e+00,0.000000e+00,6.743846e+06,6.000000e+06,0,0,0,0,0,0
4772797,41497130,2,2018-04-20,73917,12,1.0,0.0,0.000000e+00,1.0,0.0,...,0.000000e+00,0.000000e+00,1.000000e+05,8.749703e+04,0,0,0,1,0,0
412070,70628252,7,2018-10-29,21086,14,2.0,0.0,0.000000e+00,2.0,0.0,...,0.000000e+00,0.000000e+00,3.949794e+06,3.514527e+06,0,0,0,3,0,0


In [ ]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer(missing_values='np.nan', strategy = 'most_frequent')
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print(imputed_X_train.columns.isnull().sum())